In [1]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [2]:
META_TWO <- read.table("../data/EUROPEAN_GWAS_ALLELECORRECTED.TAB",header=T,fill=TRUE,row.names=NULL)

In [3]:
head(META_TWO)

,MARKERNAME,CHROMOSOME,POSITION,ADIPO_EA,ADIPO_NEA,ADIPO_EAF,ADIPO_N,ADIPO_BETA,ADIPO_SE,ADIPO_PVALUE,METSIM_EA,METSIM_NEA,METSIM_EAF,METSIM_N,METSIM_BETA,METSIM_SE,METSIM_PVALUE
,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,rs10,7,92383888,C,A,0.03333,21257,-0.004702,0.015279,0.7653763,C,A,0.07344,9236,-0.062990,0.02893,0.02947
2,rs10000012,4,1357325,G,C,0.19170,29331,-0.008930,0.006456,0.1798523,G,C,0.12227,9236,-0.004815,0.02297,0.83400
3,rs10000017,4,84778125,T,C,0.22320,29230,-0.003433,0.005454,0.5416628,T,C,0.23253,9236,0.045910,0.01781,0.00995
4,rs1000002,3,183635768,T,C,0.47500,29335,-0.002754,0.004301,0.5347318,T,C,0.39411,9236,0.006938,0.01552,0.65490
5,rs10000023,4,95733906,T,G,0.40830,29339,0.000369,0.004491,0.9365264,T,G,0.38454,9236,-0.020860,0.01533,0.17360
6,rs1000003,3,98342907,G,A,0.12500,29271,0.004954,0.005908,0.4162287,G,A,0.23338,9236,-0.002088,0.01797,0.90750


In [4]:
# First Pass RSID

In [5]:
RSID_TWO <- read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_RSIDS_FIRSTPASS.TAB",fill=TRUE,row.names=NULL)
colnames(RSID_TWO) <- "MARKERNAME"

In [6]:
## Prepare Input

In [8]:
EUR_SECONDPASS <- merge(META_TWO,RSID_TWO,by="MARKERNAME")%>%select(MARKERNAME,ADIPO_BETA,ADIPO_SE,METSIM_BETA,METSIM_SE)

In [9]:
write.table(EUR_SECONDPASS,"../data/EUROPEAN_METASOFT_SECONDPASS.TAB",row.names=FALSE,col.names=FALSE,quote=FALSE)
write.table(EUR_SECONDPASS,"../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS.TAB",row.names=FALSE,col.names=FALSE,quote=FALSE)

In [ ]:
#### Prepare Final Results 

In [10]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)

In [11]:
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")

In [12]:
nrow(Metasoft_results)

[1] 176885

In [27]:
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")

In [28]:
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)%>%filter(PVALUE_FE < 5E-08)

In [29]:
write.table(EUR_RESULTS,"../../ANALYSIS/META/EUR_RESULTS_GS.TAB",row.names=FALSE,quote=FALSE,sep="\t")

In [17]:
### GCTA input

In [19]:
colnames(EUR_RESULTS)

[1] "MARKERNAME"      "CHROMOSOME"      "POSITION"        "ADIPO_EA"       
 [5] "ADIPO_NEA"       "ADIPO_EAF"       "ADIPO_N"         "ADIPO_BETA"     
 [9] "ADIPO_SE"        "ADIPO_PVALUE"    "METSIM_EA"       "METSIM_NEA"     
[13] "METSIM_EAF"      "METSIM_N"        "METSIM_BETA"     "METSIM_SE"      
[17] "METSIM_PVALUE"   "PVALUE_FE"       "BETA_FE"         "STD_FE"         
[21] "PVALUE_RE"       "BETA_RE"         "STD_RE"          "PVALUE_RE2"     
[25] "STAT1_RE2"       "STAT2_RE2"       "PVALUE_BE"       "I_SQUARE"       
[29] "Q"               "PVALUE_Q"        "TAU_SQUARE"      "PVALUE_ADIPOGEN"
[33] "PVALUE_METSIM"   "TOTAL_N"         "MAF_AVG"

In [22]:
GCTA_INPUT <- EUR_RESULTS %>%select(MARKERNAME,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE,PVALUE_FE,TOTAL_N)%>%rename(SNP=MARKERNAME,A1=METSIM_EA,A2=METSIM_NEA,freq=MAF_AVG,b=BETA_FE,se=STD_FE,p=PVALUE_FE,n=TOTAL_N)

In [ ]:
write.table(GCTA_INPUT,"../../ANALYSIS/COJO/GCTA_EUR_INPUT.TAB",row.names=FALSE,quote=FALSE,sep="\t")